In [ ]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
!pip install protobuf==3.20


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
pip install sacremoses

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install mlflow


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
%pip install datasets   sacrebleu ipywidgets  protobuf==3.20 mlflow sacremoses  -U accelerate

Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-3.20.0-py2.py3-none-any.whl.metadata (720 bytes)
  Using cached mlflow-2.11.3-py3-none-any.whl.metadata (15 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached cloudpickle-3.0.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached importlib_metadata-7.1.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached alembic-1.13.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached docker-7.0.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl.metadata (559 bytes)
  Using cached Markdown-3.6-py3-none-any.whl.metadata (7.0 kB)
  Using cached graphene-3.3-py2.py3-none-any.whl.metadata (7.7 kB)
  Using cached waitress-3.0.0-py3-none-any

In [ ]:
pip install -U accelerate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install sacrebleu

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
os.environ["WANDB_DISABLED"]="true"

In [ ]:
import transformers
print(transformers.__version__)


4.39.1


In [ ]:
import transformers


In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-es-en"

In [ ]:
from datasets import load_dataset, load_metric


metric = load_metric("sacrebleu")


In [ ]:
from datasets import Dataset, DatasetDict, load_dataset, load_metric

datasets = load_dataset("json", data_files=r"C:\Users\iheba\Downloads\dataset translation\traductions_es_en.json")





total_size = 32000
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

train_dataset = Dataset.from_dict(datasets["train"][:train_size])
val_dataset = Dataset.from_dict(datasets["train"][train_size:train_size + val_size])
test_dataset = Dataset.from_dict(datasets["train"][train_size + val_size:train_size + val_size + test_size])


raw_datasets = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

raw_datasets


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 25600
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3200
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3200
    })
})

In [ ]:
raw_datasets["train"][0]


{'translation': {'en': 'happiness', 'es': 'felicidad'}}

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'zié diabaté', 'es': 'Zié Diabaté'}"
1,"{'en': 'lars anders tomter', 'es': 'Lars Anders Tomter'}"
2,"{'en': 'a61 autoroute', 'es': 'Autoroute A61'}"
3,"{'en': 'yechiel yaakov weinberg', 'es': 'Yechiel Yaakov Weinberg'}"
4,"{'en': 'fraxinus excelsior', 'es': 'Fraxinus excelsior'}"


In [ ]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[160, 5316, 2, 58, 60, 898, 26, 1455, 5546, 55, 0], [13357, 9, 31, 8, 346, 6545, 26, 1455, 5546, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[3923, 2, 58, 105, 6393, 55, 0], [143, 31, 826, 6393, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "es"
target_lang = "en"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[12167, 0], [4764, 4533, 0]], 'attention_mask': [[1, 1], [1, 1, 1]], 'labels': [[13185, 0], [9898, 9495, 18638, 1495, 0]]}

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/25600 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

In [ ]:
import torch
print(torch.__version__)
import transformers
print(transformers.__version__)


2.2.2+cpu
4.39.1


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"spark-name-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\iheba\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

  0%|          | 0/4800 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


{'loss': 1.0685, 'grad_norm': 6.471264839172363, 'learning_rate': 1.7916666666666667e-05, 'epoch': 0.31}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


{'loss': 0.8918, 'grad_norm': 5.951194763183594, 'learning_rate': 1.5833333333333333e-05, 'epoch': 0.62}


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


{'loss': 0.5533, 'grad_norm': 3.8324358463287354, 'learning_rate': 1.25e-06, 'epoch': 2.81}


  0%|          | 0/200 [00:00<?, ?it/s]

{'eval_loss': 0.7762151956558228, 'eval_bleu': 60.1118, 'eval_gen_len': 6.3697, 'eval_runtime': 1219.4172, 'eval_samples_per_second': 2.624, 'eval_steps_per_second': 0.164, 'epoch': 3.0}
{'train_runtime': 7850.995, 'train_samples_per_second': 9.782, 'train_steps_per_second': 0.611, 'train_loss': 0.7178738244374593, 'epoch': 3.0}


TrainOutput(global_step=4800, training_loss=0.7178738244374593, metrics={'train_runtime': 7850.995, 'train_samples_per_second': 9.782, 'train_steps_per_second': 0.611, 'train_loss': 0.7178738244374593, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


model.safetensors:   0%|          | 0.00/310M [00:00<?, ?B/s]

events.out.tfevents.1713445147.iheb-akermi.24720.0:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ihebaker10/spark-name-es-to-en/commit/9267dfc2a66d9c717fd2364aa5283d4a1ab1ae1a', commit_message='End of training', commit_description='', oid='9267dfc2a66d9c717fd2364aa5283d4a1ab1ae1a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import os
for dirname, _, filenames in os.walk('ihebaker10/spark-name-Translation-arm-en'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
src_text = ['чалэ']

model_name = 'ihebaker10/opus-mt-ru-en-finetuned-ru-to-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
print(tokenizer.supported_language_codes)


OSError: ihebaker10/opus-mt-ru-en-finetuned-ru-to-en is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
model = MarianMTModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]